In [9]:
!pip install -q -U langchain transformers bitsandbytes accelerate
!pip install langchain-community langchain-core

In [10]:
import torch
import os
from langchain import PromptTemplate, HuggingFacePipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

In [11]:
os.environ["HF_TOKEN"]='Hugging face token'

In [12]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
# MODEL_NAME = "unsloth/llama-3-8b-Instruct-bnb-4bit"
# MODEL_NAME ="mistralai/Mistral-7B-Instruct-v0.2"
# MODEL_NAME ="meta-llama/Meta-Llama-3-8B"
# MODEL_NAME ="microsoft/Phi-3-mini-4k-instruct"
# MODEL_NAME ="microsoft/phi-1_5"

# Quantization is a technique used to reduce the memory and computation requirements
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the language model,
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 8192 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 1 # randomness of the generated tex
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
# generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

In [14]:
# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [15]:
# HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [16]:
city = "los angeles"
distance = "50 miles"

input_text = f"""
  Please write all the tourist places in {city} within {distance} from nearest airport. My purpose of visit is to visit tourist places.
  I want the output in json format. Json schema to be followed:
  [{{
    place: name of the tourist spot,
    why_to_visit: tourist attraction (at least 100 words),
    best_time_to_visit_day: best time to visit during day (specific like morning, evening, night),
    best_time_to_visit_month: best month to visit to enjoy to fullest (march to april),
    distance: distance from airport in miles (approximate range),
    how_to_reach: travel options to reach this place (be specific)
  }}, .....]


  return the list of such place objects. I want at least 20 such places. also do not repeat. do not write prices and brand names. only try to write facts and give accurate data. please insert all type of places of each variety. please consider all type of places, like parks, buildings, historical monuments, natural places for scenic beauty, some other important place to visit as a tourist etc
"""

output = llm.invoke(input_text)

print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



  Please write all the tourist places in los angeles within 50 miles from nearest airport. My purpose of visit is to visit tourist places.
  I want the output in json format. Json schema to be followed:
  [{
    place: name of the tourist spot,
    why_to_visit: tourist attraction (at least 100 words),
    best_time_to_visit_day: best time to visit during day (specific like morning, evening, night),
    best_time_to_visit_month: best month to visit to enjoy to fullest (march to april),
    distance: distance from airport in miles (approximate range),
    how_to_reach: travel options to reach this place (be specific)
  }, .....]


  return the list of such place objects. I want at least 20 such places. also do not repeat. do not write prices and brand names. only try to write facts and give accurate data. please insert all type of places of each variety. please consider all type of places, like parks, buildings, historical monuments, natural places for scenic beauty, some other importa

In [17]:
city = "los angeles"
place = "Venice Beach Boardwalk"

input_text = f"""
    Get a list of hotels near {place} in city {city}. Include the following information for each hotel:

    Hotel name
    Address
    Rating (out of 5 stars)
    Distance from {place} (in miles)
    Average price per night (in USD)
    Return the list of hotels in JSON format, with the following schema:

    [ {{ "hotel_name": string, "address": string, "rating": number (out of 5), "distance": number (in miles), "average_price": number (in USD) }}, ... ]

    I want accurate outputs. please write at least 20 hotels in json
  """

output = llm.invoke(input_text)

print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    Get a list of hotels near Venice Beach Boardwalk in city los angeles. Include the following information for each hotel:

    Hotel name
    Address
    Rating (out of 5 stars)
    Distance from Venice Beach Boardwalk (in miles)
    Average price per night (in USD)
    Return the list of hotels in JSON format, with the following schema:

    [ { "hotel_name": string, "address": string, "rating": number (out of 5), "distance": number (in miles), "average_price": number (in USD) }, ... ]

    I want accurate outputs. please write at least 20 hotels in json
   */

  const hotels = [
    {
      hotel_name: "The Shore Hotel",
      address: "1515 James Woods Blvd, Venice, CA 90291",
      rating: 4.7,
      distance: 0.6,
      average_price: 329,
    },
    {
      hotel_name: "Venice Beach Inn",
      address: "400 Venice Way, Venice, CA 90291",
      rating: 4.3,
      distance: 0.5,
      average_price: 279,
    },
    {
      hotel_name: "Hotel Erwin",
      address: "1697 Pacific

In [18]:
city = "los angeles"
place = "Venice Beach Boardwalk"

input_text = f"""
    Get a list of restaurants near {place} in city {city}. Include the following information for each restaurant:

    Restaurant name Address
    Cuisine type
    Rating (out of 5 stars)
    Distance from {place} (in miles)
    Average price range (in USD)

    Return the list of restaurants in JSON format, with the following schema:
    [ {{ "restaurant_name": string, "address": string, "cuisine": string, "rating": number (out of 5), "distance": number (in miles), "average_price_range": string (e.g. "10-20") }}, ... ]

    I want accurate outputs. Please write at least 20 restaurants in JSON format
  """

output = llm.invoke(input_text)

print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    Get a list of restaurants near Venice Beach Boardwalk in city los angeles. Include the following information for each restaurant:

    Restaurant name Address 
    Cuisine type 
    Rating (out of 5 stars) 
    Distance from Venice Beach Boardwalk (in miles) 
    Average price range (in USD) 

    Return the list of restaurants in JSON format, with the following schema:
    [ { "restaurant_name": string, "address": string, "cuisine": string, "rating": number (out of 5), "distance": number (in miles), "average_price_range": string (e.g. "10-20") }, ... ]

    I want accurate outputs. Please write at least 20 restaurants in JSON format
   ```
   [
   {
      "restaurant_name":"Burger Lounge Venice",
      "address":"1303 Abbot Kinney Blvd",
      "cuisine":"American",
      "rating":4.5,
      "distance":0.2,
      "average_price_range":"10-20"
   },
   {
      "restaurant_name":"Venice Whole Foods Market",
      "address":"1010 Abbot Kinney Blvd",
      "cuisine":"Health Food",
   